In [1]:
push!(LOAD_PATH,".")

5-element Vector{String}:
 "@"
 "@v#.#"
 "@stdlib"
 "/home/skh/MultiAgentSensing"
 "."

In [2]:
using Revise
using POMDPs
using POMDPSimulators
using Dates

In [14]:
using MDMA

┌ Info: Precompiling MDMA [top-level]
└ @ Base loading.jl:1423


Test Summary: | Pass  Total
detectTarget  |   30     30
2.0
-6.0
-11.333333333333332
-15.333333333333332
-18.53333333333333
-21.2
-23.485714285714284
-25.485714285714284
-27.263492063492063
-28.863492063492064
-30.318037518037517
-31.65137085137085
-32.88214008214008
-34.02499722499723
-35.0916638916639
-36.0916638916639
-37.03284036225213
-37.92172925114102
-38.763834514298914
-39.56383451429891
-40.32573927620367
-41.0530120034764
-41.74866417738944
-42.41533084405611
-43.05533084405611
-43.67071545944072
-44.26330805203332
-44.834736623461886
-45.38646076139292
-45.91979409472625
Test Summary:        |
single_robot_planner | No tests


In [4]:
sensor = ViewConeSensor(pi/2, 3);
targets = Vector{Target}(undef, 0);
push!(targets, Target(25,25,0));
push!(targets, Target(26,25,0));
push!(targets, Target(26,26,0));
push!(targets, Target(20,5,0));

# push!(targets, Target(7,25,0));
# push!(targets, Target(8,25,0));
# push!(targets, Target(9,26,0));
# push!(targets, Target(7,24,0));
# push!(targets, Target(6,23,0));
grid = Grid(30,30);
horizon = 12
move_dist = 3
model = MDMA.SingleRobotMultiTargetViewCoverageProblem(grid, sensor, horizon, target_trajectory, move_dist);


LoadError: UndefVarError: ViewConeSensor not defined

In [5]:
policy = MDMA.solve_single_robot(model, UAVState(1,1,:N))


LoadError: UndefVarError: MDMA not defined

In [6]:
function rew_check(model, state, policy)
    prev = MDPState(state)
    next = action(policy, prev)
    rew1 = MDMA.reward(model, prev, prev.state)
    rew2 = MDMA.reward(model, prev, next)
    println(prev.state," ", rew1)
    println(next," ", rew2)
end
function check(model, prev, next)
    prev = MDPState(prev)
    rew1 = MDMA.reward(model, prev, prev.state)
    rew2 = MDMA.reward(model, prev, next)
    println(prev.state," ", rew1)
    println(next," ", rew2)
end


check (generic function with 1 method)

In [7]:
test_state = UAVState(3,3,:S)
rew_check(model, test_state, policy)

LoadError: UndefVarError: UAVState not defined

In [8]:
check(model, test_state, UAVState(3,3,:E))

LoadError: UndefVarError: UAVState not defined

In [9]:
function compute_path(model, policy, state)
    path = Vector{MDPState}(undef, 0)
    push!(path, state)
    for x in 1:model.horizon
        state = action(policy, state);
        state = MDPState(state);
        push!(path, state);
    end
    path
end
path = compute_path(model, policy, MDPState(UAVState(1,1,:S)))

LoadError: UndefVarError: UAVState not defined

In [10]:
using Cairo

#Pixels per meter
function init_cairo(model,ppm, buf)
    width = ppm*dims(model.grid)[1]+ 2*buf*ppm
    height = ppm*dims(model.grid)[2]+ 2*buf*ppm
    c = CairoRGBSurface(width,height);
    cr = CairoContext(c);
    draw_background(cr, width, height)
    return (c, cr)
end


function draw_background(cr::CairoContext, width, height)
    save(cr);
    set_source_rgb(cr,1,1,1);    # white
    rectangle(cr,0.0,0.0,width,height); # background
    fill(cr);
    restore(cr);
end

function draw_grid(g::Grid, cr::CairoContext, point_size, ppm, buf)
    set_source_rgba(cr, 0, 0, 0, 0.3);
    x,y,z = dims(g)
    print(dims(g))
    for i in 1:x
        for j in 1:y
            arc(cr, i*ppm + buf*ppm, j*ppm+ buf*ppm, point_size, 0, 2*pi);
            fill(cr);
        end
    end    
end

function draw_target(cr::CairoContext, t::Target, ppm, size, buf)
    save(cr)
    set_source_rgba(cr, 0, 0.3, 0.5, 1);
    arc(cr, t.x*ppm + buf*ppm, t.y*ppm + buf*ppm, size, 0, 2*pi);
    fill(cr);
    restore(cr)
end

function draw_targets(cr::CairoContext, targs::Vector{Target}, ppm, size, buf)
    for t in targs
        draw_target(cr, t,ppm, size, buf)
    end
end

function draw_state(cr::CairoContext, state::UAVState, model, ppm, fade, cfade, buf)
    save(cr)
    move_to(cr,state.x*ppm + buf*ppm, state.y*ppm + buf*ppm)
    fov = model.sensor.fov
    radius = model.sensor.cutoff
    draw_arc(cr, radius, state.x, state.y, state.heading, fov, ppm, fade,cfade, buf)
    restore(cr)
end
function draw_arc(cr::CairoContext, radius, x,y, heading,fov,ppm, fade,cfade, buf) 
    ## original example, following here
    xc = x*ppm + buf*ppm;
    yc = y*ppm + buf*ppm;
    radius = radius*ppm;
    angle1 = dirAngle(heading) + (-fov/2);  # angles are specified
    angle2 = dirAngle(heading) + (fov/2);  # in radians

#     set_source_rgba(cr, 0, 0, 0, fade);
    set_source_rgba(cr, (cfade), (1-cfade)*0.5, (1-cfade)*1.1, fade);
    set_line_width(cr, 5.0);
    arc(cr, xc, yc, radius, angle1, angle2);
    fill(cr)
    stroke(cr);

    # draw helping lines
#     set_source_rgba(cr, cfade, 1-cfade, 1-cfade, fade);
    set_line_width(cr, 6.0);

    # Draw center
    arc(cr, xc, yc, 10.0, 0, 2*pi);
    fill(cr);
    
    arc(cr, xc, yc, radius, angle1, angle1);
    line_to(cr, xc, yc);
#     fill(cr)
    
    arc(cr, xc, yc, radius, angle2, angle2);
    line_to(cr, xc, yc);
#     fill(cr)
    close_path(cr);
    set_source_rgba(cr, (cfade), (1-cfade)*0.5, (1-cfade)*1.1, fade/2);
    fill_preserve(cr);
    
    stroke(cr);
end


# draw_state(cr, UAVState(1,1,:S), model, ppm)


## mark picture with current date


LoadError: UndefVarError: Grid not defined

In [11]:

function draw_path(model, ppm, path, cutoff, buf)
    c, cr = init_cairo(model, ppm, buf)

    draw_grid(model.grid, cr, 5, ppm, buf)

    save(cr)
    select_font_face(cr, "Latin Modern Math", Cairo.FONT_SLANT_NORMAL,
                 Cairo.FONT_WEIGHT_NORMAL);

    set_font_size(cr, 80.0);
    set_source_rgba(cr, 0, 0, 0, 1);
    move_to(cr, model.grid.width/2*ppm + -2*ppm, model.grid.height*ppm + 1.6*buf*ppm);
    show_text(cr, "𝐷 = $(model.move_dist)  ε = 6.8E-07");
    restore(cr)
  

    # Draw States
    state=path[1]
    move_to(cr,state.state.x*ppm + buf*ppm, state.state.y*ppm + buf*ppm)
    draw_state(cr, state.state, model, ppm, 0.4, 1/cutoff, buf)
    for (i,state) in enumerate(path[2:cutoff])
        draw_state(cr, state.state, model, ppm, 0.4, i/cutoff, buf)         
    end
    
    # Draw Lines
    state=path[1]
    move_to(cr,state.state.x*ppm + buf*ppm, state.state.y*ppm + buf*ppm)
    for (i,state) in enumerate(path[2:cutoff])
        cfade = i/cutoff
        fade=1
        set_source_rgba(cr, (cfade), (1-cfade)*0.5, (1-cfade)*1.1, fade/2);
#         set_source_rgba(cr, 0.9, 0.6, 0.01, 1);
        set_line_width(cr, 13.0);
        move_to(cr,path[i].state.x*ppm+ buf*ppm, path[i].state.y*ppm + buf*ppm)
        line_to(cr, state.state.x*ppm+buf*ppm, state.state.y*ppm + buf*ppm)
        stroke(cr) 
         
    end
    
    draw_targets(cr, model.targets,ppm, 15, buf)
    write_to_png(c,"fixed_2.png");
end

draw_path (generic function with 1 method)

In [12]:
draw_path(model, 50, path, model.horizon-3, 4)

LoadError: UndefVarError: model not defined

In [13]:
draw_path(model, 50, compute_path(model, policy, MDPState(UAVState(10,10,:E)), model.horizon, 4)

LoadError: syntax: incomplete: premature end of input